In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe_geography = OneHotEncoder()
geo_df = ohe_geography.fit_transform(df[['Geography']])
geo_df

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
ohe_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_df = pd.DataFrame(geo_df.toarray(), columns = ohe_geography.get_feature_names_out(['Geography']))
geo_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [8]:
df = pd.concat([df.drop('Geography', axis = 1), geo_df], axis = 1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
with open('models/le_gender.pkl', 'wb') as file:
    pickle.dump(le_gender, file)
    
with open('models/ohe_geography.pkl', 'wb') as file:
    pickle.dump(ohe_geography, file)

In [10]:
X = df.drop('EstimatedSalary', axis=1)
y = df['EstimatedSalary']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
with open('models/reg_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [17]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', metrics=['mae'], loss = 'mean_absolute_error')

In [20]:
# Set up Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = 'reg_logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)

In [21]:
#Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True)

history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [early_stopping_callback, tensorflow_callback]
    )

Epoch 1/100
250/250 [==============================] - 3s 8ms/step - loss: 100365.2344 - mae: 100365.2344 - val_loss: 98475.3516 - val_mae: 98475.3516
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 99508.3125 - mae: 99508.3125 - val_loss: 96772.4453 - val_mae: 96772.4453
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 96588.2656 - mae: 96588.2656 - val_loss: 92532.0078 - val_mae: 92532.0078
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 90932.1719 - mae: 90932.1719 - val_loss: 85533.9766 - val_mae: 85533.9766
Epoch 5/100
250/250 [==============================] - 3s 10ms/step - loss: 82799.4922 - mae: 82799.4922 - val_loss: 76675.5312 - val_mae: 76675.5312
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 73416.1562 - mae: 73416.1562 - val_loss: 67680.4297 - val_mae: 67680.4297
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 64596.8477 - mae: 64596.8477

In [26]:
# Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
%tensorboard --logdir reg_logs/fit/

Reusing TensorBoard on port 6006 (pid 27096), started 0:00:15 ago. (Use '!kill 27096' to kill it.)

In [28]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 7ms/step - loss: 50256.2891 - mae: 50256.2891
Test MAE : 50256.2890625


In [ ]:
model.save('models/reg_model.h5')